<img style="float: right; padding: 10px;" width="200" src="images/logo-onem2m-the-iot-standard.jpg">


# oneM2M - Groups

This notebook demonstrates how oneM2M resources can be organized into groups.  
&lt;Group> resources offer a so-called *fan-out point* that can be the target to address all resources of a group.

- Create a &lt;group> resource with &lt;container> resources
- Retrieve the &lt;container> resources via the &lt;group>
- Add a &lt;contentInstance> to the &lt;group>'s containers
- Retrieve all the latest &lt;contentInstances> from the &lt;group>

## Intitialization
The section does import necessary modules and configurations, and prepares the CSE for this notebook.

In [ ]:
%run src/init.py groups

<img align="right" src="images/04/create_group.png">

## Create a &lt;group> Resource


With the following request we create a &lt;group> resource with the two &lt;container> resources as members.


<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 450px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
Note, that a &lt;group> may contain resources of mixed type. Requests to the &lt;group>'s members in general only affect fitting resources.
    </div>
</div>
</p>



In [ ]:
CREATE (                                             # CREATE request
    
    # Add the <group> under the <AE>
    to                      = cseBaseName + '/Notebook-AE',
    
    # Request Parameters
    originator              = 'Cmyself',                      # Set the originator
    requestIdentifier       = '123',                          # Unique request identifier
    releaseVersionIndicator = '3',                            # Release version indicator
    resourceType            = Type.Group,                     # Type of the resource
    
    # Request Body
    primitiveContent = 
        {
            'm2m:Group': {                                    # <container> resource
                'resourceName': 'Group',                      # Name of the resource
                'memberIDs': [                                # List of <group> member resources
                    cseBaseName + '/Notebook-AE/Container',
                    cseBaseName + '/Notebook-AE/Container_2'
                ],
                'maxNrOfMembers': 10                          # Maximum number of members is 10
            }
        }
)

<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 450px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
    In addition to the &lt;group> resource the CSE provides a virtual <i>fopt</i> (fan-out point) resource under the &lt;group> resource. Requests to this virtual resource are multiplied by the CSE and sent to each of &lt;group>'s member resources. The full response messages of individual requests, including for example the Response Status Codes, are collected and returned in the response to original request.
    <br><br>
Also note, that the resource identifiers in the group's <i>mid</i> attribute have been changed to unstructured format.     </div>
</div>
</p>
 
 
### Requests & Response Attributes

The request and response introduce new attributes:

| Attribute           | Short Name | Description                                                                                                                                   |
|:--------------------|:-----------|:----------------------------------------------------------------------------------------------------------------------------------------------|
| memberIDs           | mid        | The resource IDs of the group's members.                                                                                                      |
| maxNrOfMembers      | mnm        | The maximum number of member of a &lt;group>.                                                                                                 |
| memberType          | mt         | This is the resource type of the member resources. The member type is *mixed* (0) in case the member resourtces are not all of the same type. |
| currentNrOfMembers  | cnm        | The current number of member of the &lt;Group> resource.                                                                                      |
| memberTypeValidated | mtv        | Member types validated. This read-only attribute indicates whether the hosting CSE correctly validated the member resources.                  |
| consistencyStrategy | csy        | The consistency strategy determines how the CSE deals with the &lt;group> resource if the member type validation fails.                       |



<img align="right" src="images/04/retrieve_containers.png">

## Retrieve all &lt;container> Resources of a &lt;group> Resource


The following request retrieves all the &lt;container> resources that are members of the &lt;group> resource.  
The request targets the virtual *fan-out point* child resource of the &lt;group> resource.


In [ ]:
RETRIEVE (                                          # RETRIEVE request

    # Retrieve the container resources from the group
    to                      = cseBaseName + '/Notebook-AE/Group/fopt', 

    # Request Parameters
    originator              = 'Cmyself',            # Set the originator
    requestIdentifier       = '123',                # Unique request identifier
    releaseVersionIndicator = '3',                  # Release version indicator
)


<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 450px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
The response contains a list of aggregated responses from the individual RETRIEVE requests that are fanned-out to the member resources of the &lt;group>.
    </div>
</div>
</p>
<img align="right" src="images/04/create_contentInstances.png">

 
### Requests & Response Attributes

The request and response introduce new attributes:

| Attribute          | Short Name | Description                                                                                      |
|:-------------------|:-----------|:-------------------------------------------------------------------------------------------------|
| aggregatedResponse | agr        | An aggregated response contains a collection of individual response primitives.                  |
| responsePrimitive  | rsp        | A single response to a request. It contains various response attributes and a primitive content. |
| primitiveContent   | pc         | The the actual resource or other answer to a request.                                            |



## Add &lt;contentInstance> Resources to all the &lt;group> Members


The following request creates &lt;contentInstance> resources, one under each member of the &lt;group> resource.  
Again, this request targets the virtual *fan.out point* child resource of the &lt;group> resource.

In [ ]:
CREATE (                                             # CREATE request
    
    # Send the request to the <fopt> virtual resoure
    to                      = cseBaseName + '/Notebook-AE/Group/fopt',
    
    # Request Parameters
    originator              = 'Cmyself',             # Set the originator
    requestIdentifier       = '123',                 # Unique request identifier
    releaseVersionIndicator = '3',                   # Release version indicator
    resourceType            = Type.ContentInstance,  # Type of the resource
    
    # Request Body
    primitiveContent = 
        {
            'm2m:ContentInstance': {
                'contentInfo': 'text/plain:0',
                'content': 'Hello, World!'
            }
        }
)


<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 450px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
The request returns the created &lt;contentInstance> resources from both the &lt;container> resources inside the aggregated response.
    </div>
</div>
</p>


<img align="right" src="images/04/retrieve_latest.png">


## Retrieve the latest \<contentInstance> Resources from the \<container> Resources

With the following request we will retrieve all the latest &lt;contentInstances> from the &lt;group>.

<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 450px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
    We target the &lt;group>'s &lt;container> resources normally via the <i>fopt</i> virtual resource and then extend the path by appending <i>la</i> for the &lt;latest> virtual resource to access the latest &lt;contentInstance> resources. This path is internally extened by the CSE for each of the &lt;group>'s members.
    </div>
</div>
</p>

In [ ]:
RETRIEVE (                                          # RETRIEVE request

    # Retrieve the latest content instance resource from all the group's containers
    to                      = cseBaseName + '/Notebook-AE/Group/fopt/la',

    # Request Parameters
    originator              = 'Cmyself',            # Set the originator
    requestIdentifier       = '123',                # Unique request identifier
    releaseVersionIndicator = '3',                  # Release version indicator
)

&nbsp;